In [ ]:
# script to replace the base mechanism with new reactions (every reaction calculated) and species (all species calculated for now)

In [1]:
import os
import datetime
import subprocess
import numpy as np

import rmgpy.chemkin
import rmgpy.data.kinetics
# import cantera as ct


import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_kinetics(rxns, labels=None):
    """Function for plotting reaction kinetics
    Takes in a list of RMG reactions (rmgpy.reaction.Reaction) or a single reaction
    """
    plt.xlabel('1000 / T (K^-1)')
    plt.ylabel('ln(k)')

    if type(rxns) != list:
        rxns = [rxns]
    
    T = np.linspace(300, 3000, 1001)
    for rxn in rxns:
        k = np.zeros(len(T))
        for i in range(0, len(T)):
            k[i] = rxn.get_rate_coefficient(T[i], 101325)
        plt.plot(1000.0 / T, np.log(k))

    if labels:
        plt.legend(labels)
    plt.show()

In [ ]:
def plot_thermos(thermos, labels=None):
    %matplotlib inline
    fig, ax = plt.subplots(1,3)
    fig.set_size_inches(12, 3)
    fig.tight_layout()
    ax[0].set_xlabel('Temperature (K)')
    ax[0].set_ylabel('H (kJ / mol)')
    ax[0].set_title('Enthalpy vs. Temperature')
    ax[1].set_xlabel('Temperature (K)')
    ax[1].set_ylabel('S (kJ / mol K)')
    ax[1].set_title('Entropy vs. Temperature')
    ax[2].set_xlabel('Temperature (K)')
    ax[2].set_ylabel('Cp (kJ / mol K)')
    ax[2].set_title('Heat Capacity vs. Temperature')
    T = np.linspace(300, 3000, 1001)
    for thermo in thermos:
        H = np.zeros(len(T))
        S = np.zeros(len(T))
        Cp = np.zeros(len(T))
        for i in range(0, len(T)):
            H[i] = thermo.get_enthalpy(T[i]) / 1000.0
            S[i] = thermo.get_entropy(T[i]) / 1000.0
            Cp[i] = thermo.get_heat_capacity(T[i]) / 1000.0
        ax[0].plot(T, H)
        ax[1].plot(T, S)
        ax[2].plot(T, Cp)
    ax[0].legend(labels)
    ax[1].legend(labels)
    ax[2].legend(labels)
    plt.subplots_adjust(wspace=0.25)
    plt.show()

In [18]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model'
# basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24'
# basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_small_core'
# new_model_dir = '/work/westgroup/harris.se/autoscience/autoscience/butane/improved_models'
new_model_dir = basedir
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')


110 species, 1822 reactions


In [5]:
# Load the new kinetics library
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
kinetics_lib = os.path.join(DFT_DIR, 'kinetics', 'kinetics')
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(kinetics_lib)
print(len(ark_kinetics_database.libraries[''].entries), 'reactions')

33 reactions


In [14]:
# change the 4 reactions
reactions_to_change = [804, 805, 808, 809]
for i in reactions_to_change:
    reaction_list[i].kinetics = ark_kinetics_database.libraries[''].entries[805].data


# chemkin_file = os.path.join(new_model_dir, f'only_4rxns_20221006.inp')
# rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
# subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])


In [15]:

# stitch each of the library reactions into the mechanism (depends on library index matching indexing in chemkin file)
for key in ark_kinetics_database.libraries[''].entries.keys():
    entry = ark_kinetics_database.libraries[''].entries[key]
    index = entry.index
#     print(index, '\t', entry.label, '\t', reaction_list[index])
    for i in range(0, len(reaction_list)):
        if entry.item.is_isomorphic(reaction_list[i]):
            rmg_rxn_index = i
            break
    else:  
        print(f'could not match reaction index {index}')
        continue
          
    # change every possible reaction
    print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
    reaction_list[rmg_rxn_index].kinetics = entry.data

changing reaction 18 HO2(16) + HO2(16) <=> O2(2) + H2O2(17)
changing reaction 127 OH(15) + CH2O(9) <=> H2O(8) + HCO(19)
changing reaction 180 H2(13) + CH(3) <=> H(14) + CH2(23)
changing reaction 211 H2O2(17) + C2H3(22) <=> HO2(16) + C2H4(11)
changing reaction 247 OH(15) + butane(1) <=> H2O(8) + SC4H9(183)
changing reaction 278 O2(2) + SC4H9(183) <=> HO2(16) + C4H8-2(189)
changing reaction 288 OH(15) + butane(1) <=> H2O(8) + PC4H9(182)
changing reaction 294 CH2O(9) + PC4H9(182) <=> HCO(19) + butane(1)
changing reaction 313 O2(2) + PC4H9(182) <=> HO2(16) + C4H8-1(188)
changing reaction 324 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225)
changing reaction 370 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
changing reaction 371 CH3CHO(35) + PC4H9(182) <=> CH2CHO(21) + butane(1)
changing reaction 419 HO2(16) + CH2CHO(21) <=> H2O2(17) + CH2CO(24)
changing reaction 422 HO2(16) + SC4H9(183) <=> H2O2(17) + C4H8-1(188)
changing reaction 591 CH3CHO(35) + C3H5-A(94) <=> CH3CO(20) + C3H6(12)
changing r

# 

In [7]:
# Load the new thermo library
thermo_lib = os.path.join(DFT_DIR, 'thermo', 'thermo')
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.load_libraries(thermo_lib)
print(f'{len(ark_thermo_database.libraries["harris_butane"].entries)} entries')

110 entries


In [19]:
# stitch each of the library thermos into the mechanism

change_index = [85]

for key in ark_thermo_database.libraries['harris_butane'].entries.keys():
    entry = ark_thermo_database.libraries['harris_butane'].entries[key]
#     print(entry)

    for i, species in enumerate(species_list):
#         if entry.item.is_isomorphic(species.molecule[0]):
        if entry.item.smiles == species.smiles:
            rmg_species_index = i
            break
    else:
        entry_sp = rmgpy.species.Species(smiles=entry.item.smiles)
        for j, species in enumerate(species_list):
            if entry_sp.is_isomorphic(species.molecule[0]):
                rmg_species_index = i
                'matched'
                break
        else:
#             print(f'could not match species index {entry}')
            continue
            raise ValueError (f'could not match species index {entry}')

    if rmg_species_index in change_index:
#         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
        print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
        species_list[rmg_species_index].thermo = entry.data

        
        
#     # only change GAV entries
#     if 'library' not in species_list[rmg_species_index].thermo.comment.lower():
#         print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
# #         try:
# #             display(species_list[rmg_species_index])
# #         except:
# #             pass
# #         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
#         species_list[rmg_species_index].thermo = entry.data
# #         print(rmg_species_index)
        
        
        
#     # change everything
#     print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
#     if rmg_species_index == 85:
#         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
#     species_list[rmg_species_index].thermo = entry.data

    
    # only change GAV
    
    
#     if rmg_species_index in plot_index:
#         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
#         print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
#         species_list[rmg_species_index].thermo = entry.data

#     break
#     print(species_list[rmg_species_index])
#     print(f'Changing reaction {rmg_rxn_index}')
#     if rmg_rxn_index == 288:
#         reaction_list[rmg_rxn_index].kinetics = entry.data
# #     break
#     # compare the kinetics
#     plot_kinetics([reaction_list[rmg_rxn_index], entry.data], ['RMG', 'DFT'])
    

Changing estimated thermo 85: CC(CCOO)OO(787)


In [20]:
# custom save
# chemkin_file = os.path.join(new_model_dir, f'all_new_calcs20221006_8XX.inp')
chemkin_file = os.path.join(new_model_dir, f'only85.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])
# os.remove(chemkin_file)


Wrote CTI mechanism file to '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model/only85.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.


CompletedProcess(args=['ck2cti', '--input=/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model/only85.inp', '--transport=/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model/tran.dat', '--output=/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model/only85.cti'], returncode=0)

In [ ]:
# save the resulting mechanism
timestamp_str = datetime.datetime.now().isoformat()[0:10]
chemkin_file = os.path.join(new_model_dir, f'chem_{timestamp_str}.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={new_model_dir}/chem_{timestamp_str}.cti'])
# os.remove(chemkin_file)



In [ ]:
reaction_list[749].kinetics

In [ ]:
entry.data

In [ ]:
entry.item.is_isomorphic(reaction_list[749])

In [ ]:
dir(entry)

In [ ]:
ark_kinetics_database.libraries[''].entries

In [ ]:
species_list[96].smiles

In [ ]:
for i, rxn in enumerate(reaction_list):
    for sp in rxn.products + rxn.reactants:
        if sp.smiles == species_list[96].smiles:
            print(i, rxn)
            break

In [ ]:
reaction_list[1246].products[0].thermo

In [ ]:
reaction_list[1246].products[0].thermo

In [ ]:
reaction_list[1246].products[0].thermo